In [ ]:
from numpy import array
from keras.preprocessing.text import one_hot
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers.core import Activation, Dropout, Dense
from keras.layers import Flatten, LSTM
from keras.layers import GlobalMaxPooling1D
from keras.models import Model
from keras.layers.embeddings import Embedding
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.layers import Input
from keras.layers.merge import Concatenate

import pandas as pd
import numpy as np
import re

In [ ]:
pip install hazm

     |████████████████████████████████| 316 kB 4.2 MB/s 
     |████████████████████████████████| 233 kB 47.6 MB/s 
     |████████████████████████████████| 1.4 MB 42.3 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394487 sha256=fc27a71ab466a2ebdea3eb63727504cbd416482d20878c89b75b489a145cd1e7
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=154693 sha256=5d03e227023ee00b251f3083fd4c01d54e1e9cb7d41266ca1dcc19c7f996d862
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.2.5
    Uninstalling nltk-3.2.5:
      Successfully uninstalled nltk-3.2.5


In [ ]:
from __future__ import print_function, unicode_literals
import codecs, subprocess, random
from collections import Counter
from itertools import islice
from nltk.tag import untag
from sklearn.model_selection import train_test_split
from hazm import *
from hazm.Chunker import tree2brackets
from hazm.PeykareReader import coarse_pos_e as peykare_coarse_pos_e
from hazm.DadeganReader import coarse_pos_e as dadegan_coarse_pos_e
from google.colab import drive
import pandas as pd
import numpy as np
import pickle
import nltk
import glob
import re
import os
import io

drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
stop_words = pd.read_json('/content/drive/MyDrive/Colab Notebooks/emotion_project/stopwords-fa.json', encoding= 'utf-8')
stopwords=stop_words.transpose()
stop_words=stopwords.values.tolist()
stopwords[1]='!!'
stop_words=stopwords.values.tolist()
ss=stopwords.iloc[0,9:798]
sss=ss.values.tolist()

In [ ]:
!pip install emojis
import emojis
import re
import string
from string import digits
def text_cleaner(text):

  # Emoji parsing
  text = emojis.decode(str(text))
  
  # Removing numbers
  text = re.sub(r'\\d+', '', text)

  # Removing single character words
  text =  re.sub(r"\b[a-zA-Z]\b", "", text)

  # Slang parsing
  # for slang, meaning in slangs.items():
  #   text = text.replace(slang, meaning)
  
  text=text.replace(":", "")
  text=text.replace('@', "")
  text=text.replace("#", " ")
  text=text.replace("_", " ")
  text=text.replace("،", " ")
  text=text.replace("\u200c", " ")
  text =  re.sub(r"[a-zA-Z]", "", text)
  
  # Removing links
  text=text.replace(r"http\S+", "")
  text=text.translate(str.maketrans('', '', string.punctuation))
  text=text.translate(str.maketrans('', '', string.digits))
  # text = text.translate(None, digits)

  # Normalizing Case
  # Filter Out Punctuation
  # Filter out Stop Words (and Pipeline)

  # words = [w for w in text.split() if w.isalpha() and not w in sss]
  
  words = [w for w in text.split()if not w in sss]
  return ' '.join(words)

In [ ]:
#clean data adn stemming
import pandas as pd
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/Emotion.csv', encoding= 'utf-8')
df.loc[0,'text']
clean_text=[]
stemmer = Stemmer()
for i in range(len(df)) :
  text=text_cleaner(df.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text.append(t)
# clean_text[0]
len(clean_text)

5999

In [ ]:
df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/emotion_project/EmotionTest.csv', encoding= 'utf-8')
clean_text_test=[]
stemmer = Stemmer()
for i in range(len(df_test)) :
  text=text_cleaner(df_test.loc[i,'text'])
  # text=df.loc[i,'text']
  wr=[stemmer.stem(w) for w in text.split()]
  t=' '.join(wr)
  clean_text_test.append(t)
len(clean_text_test)

1000

In [ ]:
y=df.loc[:,'label']
y.replace('\n','')
y=y.values.tolist()
# x=df_featurs.values.tolist()
for i in range(len(y)):
  y[i]=y[i].replace('\n','')
  y[i]=y[i].replace('.','')
label=[]
for i in range(len(y)):
  yy = [int(t) for t in y[i].split(' ')]
  label.append(yy)
label=np.array(label)

In [ ]:

y_test=df_test.loc[:,'label']
y_test.replace('\n','')
y_test=y_test.values.tolist()
# x_test=df_featurs2.values.tolist()
for i in range(len(y_test)):
  y_test[i]=y_test[i].replace('\n','')
  y_test[i]=y_test[i].replace('.','')

ytest=[]
for i in range(len(y_test)):
  yy = [int(t) for t in y_test[i].split(' ')]
  ytest.append(yy)
ytest=np.array(ytest)

In [ ]:
tokenizer = Tokenizer(num_words=2000)
tokenizer.fit_on_texts(clean_text)

X_train = tokenizer.texts_to_sequences(clean_text)
X_test = tokenizer.texts_to_sequences(clean_text_test)

vocab_size = len(tokenizer.word_index) + 1

maxlen = 200

X_train = pad_sequences(X_train, padding='post', maxlen=maxlen)
X_test = pad_sequences(X_test, padding='post', maxlen=maxlen)

In [ ]:
from numpy import array
from numpy import asarray
from numpy import zeros

embeddings_dictionary = dict()

glove_file = open('/content/drive/MyDrive/Colab Notebooks/emotion_project/vocab_embed.txt', encoding="utf8")

for line in glove_file:
    records = line.split()
    word = records[0]
    vector_dimensions = asarray(records[1:], dtype='float32')
    embeddings_dictionary[word] = vector_dimensions
glove_file.close()

embedding_matrix = zeros((vocab_size, 100))
for word, index in tokenizer.word_index.items():
    embedding_vector = embeddings_dictionary.get(word)
    if embedding_vector is not None:
        embedding_matrix[index] = embedding_vector

In [ ]:
deep_inputs = Input(shape=(maxlen,))
embedding_layer = Embedding(vocab_size, 100, weights=[embedding_matrix], trainable=False)(deep_inputs)
LSTM_Layer_1 = LSTM(128)(embedding_layer)
dense_layer_1 = Dense(10, activation='sigmoid')(LSTM_Layer_1)
model = Model(inputs=deep_inputs, outputs=dense_layer_1)

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['acc'])
print(model.summary())
history = model.fit(X_train, label, batch_size=128, epochs=10, verbose=1, validation_split=0.2)

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200)]             0         
_________________________________________________________________
embedding (Embedding)        (None, 200, 100)          1757700   
_________________________________________________________________
lstm (LSTM)                  (None, 128)               117248    
_________________________________________________________________
dense (Dense)                (None, 10)                1290      
Total params: 1,876,238
Trainable params: 118,538
Non-trainable params: 1,757,700
_________________________________________________________________
None
Epoch 1/10
38/38 [==============================] - 43s 662ms/step - loss: 0.5413 - acc: 0.1568 - val_loss: 0.3346 - val_acc: 0.1550
Epoch 2/10
38/38 [==============================] - 24s 643ms/step - loss: 0.3319 - acc: 0.1751 - val_los

In [ ]:
score = model.evaluate(X_test, ytest, verbose=1)

print("Test Score:", score[0])
print("Test Accuracy:", score[1])
len(X_test)

32/32 [==============================] - 2s 61ms/step - loss: 0.3347 - acc: 0.1180
Test Score: 0.3347246050834656
Test Accuracy: 0.11800000071525574


1000

In [ ]:

yhat=model.predict(X_test)
for i in range(len(yhat)):
  for j in range(10):
    if yhat[i][j] <0.5 :
      yhat[i][j]=0
    else:
      yhat[i][j]=1


In [ ]:
from sklearn.metrics import hamming_loss, accuracy_score
print(hamming_loss(ytest, yhat))

0.1226
